In [1]:
import traci
import pandas as pd
import random
import time
import pickle

In [7]:
START = 23400
END = 32400
INTERVAL = 60

sumo_cmd = ["/opt/ohpc/pub/apps/sumo/bin/sumo"]
sumo_config = ["-c", "./Sim/network-background.sumocfg"]
sumo_cmd.extend(sumo_config)

In [ ]:
start_t = time.time()
traci.start(sumo_cmd, label=str(random.randint(10000, 50000)))
edge_ids = traci.edge.getIDList()

log = {}
for edge_id in edge_ids:
    log[edge_id] = {}

while traci.simulation.getTime() <= 32400:
    step = int(traci.simulation.getTime())
    if step % 100 == 0:
        print(step)
    
    if step % INTERVAL == 0:
        for edge in edge_ids:
            log[edge][step] = {
                'HaltingNumber': traci.edge.getLastStepHaltingNumber(edge),'Length': traci.edge.getLastStepLength(edge),
                'Speed': traci.edge.getLastStepMeanSpeed(edge), 'Occupancy': traci.edge.getLastStepOccupancy(edge),
                'VehicleNumber': traci.edge.getLastStepVehicleNumber(edge)
            }

    traci.simulationStep()
    
finish_t = time.time()

In [ ]:
pickle.dump(log, open('output/full_edge_data.pkl', 'wb'))
print("Sim time: ", finish_t - start_t)

In [2]:
edge_data = log

In [3]:
for edge in edge_data:
    step = START + INTERVAL
    while step <= END:
        if edge_data[edge][step]['Speed'] == edge_data[edge][step - INTERVAL]['Speed']:
            edge_data[edge][step]['will_remove'] = True
        step += INTERVAL
    step = START
    while step <= END:
        if 'will_remove' in edge_data[edge][step]:
            del edge_data[edge][step]
        step += INTERVAL

In [12]:
step = START
edge_data_by_step = {}
while step <= END:
    edge_data_by_step[step] = []
    for edge in edge_data:
        if step in edge_data[edge]:
            edge_data_by_step[step].append({'edge_id': edge, 'edge_speed': edge_data[edge][step]['Speed']})
    step += INTERVAL

In [ ]:
pickle.dump(edge_data_by_step, open('output/filtered_edge_data_by_step.pkl', 'wb'))